In [1]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langgraph.graph import StateGraph, START, END
from typing import TypedDict
from dotenv import load_dotenv

In [2]:
load_dotenv()

True

In [3]:
model = ChatGoogleGenerativeAI(model="gemini-2.5-flash")

In [20]:
class BlogState(TypedDict):
    topic: str
    outline: str
    content: str
    scorebyAi: str

In [19]:
def create_outline(state: BlogState) -> BlogState:
    
    # Extract the topic from the state
    topic = state["topic"]

    # Create a prompt for the model
    prompt = f"Create a detailed outline for a blog post about {topic}."

    # Get the outline from the model
    outline = model.invoke(prompt).content

    # Update the state with the generated outline
    state["outline"] = str(outline)

    return state

def create_blog_content(state: BlogState) -> BlogState:
    # Extract the outline from the state
    outline = state["outline"]

    # Create a prompt for the model
    prompt = f"Write a detailed blog post based on the following outline:\n{outline}"

    # Get the blog content from the model
    content = model.invoke(prompt).content

    # Update the state with the generated content
    state["content"] = str(content)

    return state

def rate_blog(state:BlogState)-> BlogState:
    # Extract the blog content
    blog_content = state["content"]
    blog_outline = state["outline"]

    # Create the prompt
    prompt = f""" 
    Rate the blog by analyzing the oultline and the content.
    Give a score from 1 to 10, where 1 is the lowest and
    10 is the highest. Provide only the score as an integer.

    Outline: {blog_outline}
    Content: {blog_content}

    """

    # Get the score from the model
    score = model.invoke(prompt).content
    state["scorebyAi"] = str(score)
    return state


In [21]:
graph = StateGraph(BlogState)

# define the nodes
graph.add_node("create_outline",create_outline)
graph.add_node("create_blog_content",create_blog_content)
graph.add_node("rate_blog", rate_blog)
# define the edges

graph.add_edge(START, "create_outline")
graph.add_edge("create_outline", "create_blog_content")
graph.add_edge("create_blog_content", "rate_blog")
graph.add_edge("rate_blog", END)


# compile the graph
workflow = graph.compile()

In [22]:
inital_state: BlogState = {"topic": "let var and const in javascript", "outline": "", "content": "","scorebyAi":""}
final_state = workflow.invoke(inital_state)
print(final_state)

{'topic': 'let var and const in javascript', 'outline': 'Here\'s a detailed outline for a blog post about `let`, `var`, and `const` in JavaScript, designed to be comprehensive and easy to understand for developers of all levels.\n\n---\n\n## Blog Post Outline: Demystifying JavaScript Variables: `let`, `var`, and `const` Explained\n\n**Target Audience:** Junior to mid-level JavaScript developers, or anyone looking for a clear understanding of variable declarations in modern JS.\n\n**Goal:** To clearly explain the differences between `var`, `let`, and `const`, including scope, hoisting, and re-declaration/re-assignment, and to provide best practices for their use.\n\n---\n\n### I. Title Options (Choose one)\n\n*   The Definitive Guide to `let`, `var`, and `const` in JavaScript\n*   Beyond `var`: Understanding `let` and `const` for Modern JavaScript\n*   Mastering JavaScript Variables: A Deep Dive into `let`, `var`, and `const`\n*   JavaScript Variables Explained: Scope, Hoisting, and the

In [23]:
print(final_state["scorebyAi"])

10
